[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sensioai/dl/blob/master/frameworks/exercise.ipynb)

# Frameworks - exercise

Using Pytorch, code a Keras-like API to train a sequential model on the MNIST dataset.

In [ ]:
import torch
from fastprogress import master_bar, progress_bar

device = "cuda" if torch.cuda.is_available() else "cpu"

class MyModel():
  def __init__(self, net):
    # save the net to use across the model

  def compile(self, loss, optimizer, metrics):
    # save the loss, optimizer and metrics to use across the model
    
  def train(self, dataloader):    
    # iterate through the dataloader optimizing the network
    # and computing the loss and metrics
      
  def eval(self, dataloader):    
    # iterate through the dataloader computing the loss and metrics
    
  def fit(self, dataloader, val_dataloader, epochs=100):
    # iterate for some epochs, training and evaluating the network
    # return an object with the learning statistics
    
  def predict(self, dataloader):    
    # iterate through the dataloader and return predictions

  def evaluate(self, dataloader):    
    # iterate through the dataloader and compute loss and metrics
    # (you can call the `eval` function)

In [ ]:
class Accuracy():
  def __init__(self):
    self.name = "acc"
  
  def __call__(self, output, labels):
    # compute and return the accuracy 

In [ ]:
class MyDataset():
    def __init__(self, images, labels=None, train=True):
        self.train = train
        self.images = images
        self.labels = labels
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, ix):
        # return and image and a label (training and evluation)
        # return only an image for testing
        
        # remember to normalize images !

The following code should work

In [ ]:
# download dataset

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import numpy as np

X, y = fetch_openml('mnist_784', version=1, return_X_y=True)

y = y.astype(np.int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_eval, y_train, y_eval = train_test_split(X_train, y_train, test_size=0.2)

In [ ]:
# define datasets and dataloaders

from torch.utils.data import DataLoader

datasets = {
    'train': MyDataset(X_train, y_train),
    'eval': MyDataset(X_eval, y_eval),
    'test': MyDataset(X_test, train=False)
}

bs = 32
dataloaders = {
    'train': DataLoader(datasets['train'], shuffle=True, batch_size=bs),
    'eval': DataLoader(datasets['eval'], shuffle=False, batch_size=bs),
    'test': DataLoader(datasets['test'], shuffle=False, batch_size=bs)
}

In [ ]:
# train the network

net = torch.nn.Sequential(
  torch.nn.Linear(28*28, 100),
  torch.nn.ReLU(),
  torch.nn.Linear(100, 10)    
)

model = MyModel(net)

model.compile(loss=torch.nn.CrossEntropyLoss(),
              optimizer=torch.optim.SGD(net.parameters(), lr=0.01),
              metrics=[Accuracy()])

history = model.fit(dataloaders['train'], dataloaders['eval'], epochs=10)

In [ ]:
# plot learning curves

import pandas as pd
import matplotlib.pyplot as plt

pd.DataFrame(history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

In [ ]:
# evaluate the model

model.evaluate(dataloaders['eval'])

In [ ]:
# get new predictions

y_proba = model.predict(dataloaders['test'])
y_pred = torch.argmax(y_proba, axis=1)

In [ ]:
# visualize some new predictions

import random

r, c = 5, 8
plt.figure(figsize=(c*1.2, r*1.2))
for row in range(r):
    for col in range(c):
        plt.subplot(r, c, c*row+col+1)
        ix = random.randint(0, len(datasets['test']))
        img = datasets['test'][ix]
        plt.imshow(img.reshape(28,28), cmap="binary", interpolation="nearest")
        plt.axis('off')
        pred = y_pred[ix].item()
        plt.title(pred, fontsize=12)
plt.tight_layout()
plt.show()